# Make your own custom analysis

In this example, a user can follow the instructions in /feasst/plugin/example/include/analyze_example.h to make their own custom analysis.
In some cases, it is preferable to implement the analysis within FEASST rather than analyze stored configurations in order to limit the number of configurations that need to be saved to disk.

In this relatively trivial example, the geometric center of all (PBC wrapped) sites is computed for an ideal gas fluid.
The PBCs are centered on the origin, so the result should be the origin, within statistical error.

In [1]:
import sys
import subprocess
import unittest
import pandas as pd

def run_fst():
    with open("launch.txt", "w") as myfile: myfile.write("""
MonteCarlo
Configuration particle_type0 /feasst/forcefield/atom.fstprt cubic_side_length 8 add_particles_of_type0 100
Potential Model IdealGas
ThermoParams beta 1
Metropolis num_trials_per_iteration 1e2
TrialTranslate tunable_param 4,
AnalyzeExample trials_per_write 1e3 file_name ig_center.csv start_after_iteration 1
Run num_trials 1e5
""")
    syscode = subprocess.call("../../../build/bin/fst < launch.txt > launch.log", shell=True, executable='/bin/bash')
    if syscode > 0: sys.exit(1)

Run the test and check the energy.

In [2]:
class TestAnalyzeText(unittest.TestCase):
    def test_analyze(self):
        run_fst()
        df = pd.read_csv('ig_center.csv')
        df['center_is_origin'] = abs(df['average']) < 3*df['block_stdev']
        self.assertTrue(df['center_is_origin'].all())
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_analyze (__main__.TestAnalyzeText) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.522s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!